<a href="https://colab.research.google.com/github/aritanaoya/keibaAI/blob/main/scrape_race_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# keiba AI

 **このAIでは以下のデータをスクレイピングして推測を行います** 




1.   テストデータ

    - 出走する馬の成績データ
        
        出馬表の各馬にhorseIDがついているのでそのデータを取得する

    - 出走する馬の血統データ
        
        上記horseIDを用いて取得

    - 最終追い切りの調教タイム
        
2. 　訓練データ

    - 推測したいレースの過去10年分の結果
        
        NetKeibaの過去のレース結果の欄には

        ```
        <td class="race_name Txt_Bold Txt_L">
            <a href="https://db.netkeiba.com/race/202009020611/">桜花賞</a>
        ```
        のようにraceIDが記載されたtableが存在するのでそのraceIDを取得してそのIDを元にレース結果を取得する

        また上記類似して以下の二点も取得
        - 過去10年分の馬の成績データ
        - 過去10年分の馬の血統

    - 推測したいレースの参考レースのデータ
    　Netkeibaに記載があるのでレースIDは手動で取得

    - 過去1年分の同じ競馬場での成績





## import modules

In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen

## スクレイピング定義

In [ ]:
def scrape_race_results(race_id_list, pre_race_results={}):
    race_results = pre_race_results
    for race_id in tqdm(race_id_list):
        if race_id in race_results.keys():
            continue
        try:
          time.sleep(0.5)
          url = 'https://db.netkeiba.com/race/' + race_id
          race_results[race_id] = pd.read_html(url)[0]
        except IndexError:
            continue
        except Exception as e:
	          print(e)
	          break
        except:
	          break
    return race_results


## raceIDのurl情報
* 年次
* 場所
```
01: 札幌 02: 函館 03: 福島 04: 新潟 05: 東京06:中山07: 中京 08: 京都 09:阪神 10: 小倉
```
* レースの開催回数
* 開催日
* round

In [ ]:
#レースIDのリストを作る
race_id_list = []
for place in range(5 ,9, 1):
  for kai in range(1, 6, 1):
    for day in range(1, 13, 1):
      for r in range(1, 13, 1):
        race_id = "2020" + str(place).zfill(2) + str(kai).zfill(2) +\
		    str(day).zfill(2) + str(r).zfill(2)
        race_id_list.append(race_id)


In [ ]:
#スクレイピングしてデータを保存
test3 = scrape_race_results(race_id_list)
for key in test3:
    test3[key].index = [key] * len(test3[key])
results = pd.concat([test3[key] for key in test3], sort=False) 
results.to_pickle('results.pickle')

In [ ]:
df= pd.read_pickle('results.pickle')
df.head(30)

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師
202005010101,1,2,2,クリアミッション,牡3,56.0,大庭和弥,1:20.4,NaN,9.4,5.0,506(+2),[東] 土田稔
202005010101,2,1,1,エストラード,牡3,56.0,江田照男,1:20.4,アタマ,8.3,4.0,478(+2),[東] 武市康男
202005010101,3,5,9,マッチャパフェ,牝3,54.0,武藤雅,1:20.4,アタマ,10.5,6.0,446(-4),[東] 林徹
202005010101,4,3,4,コルニリア,牡3,56.0,嘉藤貴行,1:20.6,3/4,19.3,9.0,440(+2),[東] 大和田成
202005010101,5,7,12,ローズオブシャロン,牝3,54.0,石橋脩,1:20.6,クビ,7.3,3.0,458(0),[東] 大竹正博
202005010101,6,3,5,バーンパッション,牡3,56.0,大野拓弥,1:20.7,クビ,20.2,10.0,468(+2),[東] 中川公成
202005010101,7,6,10,ユタカサン,牡3,56.0,吉田豊,1:20.7,アタマ,17.4,8.0,486(+8),[東] 竹内正洋
202005010101,8,4,7,サイモンバトラー,牡3,56.0,田辺裕信,1:20.7,ハナ,15,7.0,462(+12),[東] 矢野英一
202005010101,9,4,6,コパノラクラク,牝3,54.0,宮崎北斗,1:20.8,1/2,97.2,12.0,442(+4),[東] 和田勇介
202005010101,10,8,14,コーラルティアラ,牝3,54.0,横山典弘,1:20.8,アタマ,3.4,1.0,460(-2),[東] 和田勇介


In [ ]:
df_1=df[ df.index > '202006050810' ]
df_1.head(16)

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師
202006050811,1,5,9,クロノジェネシス,牝4,55.0,北村友一,2:35.0,NaN,2.5,1.0,474(+10),[西] 斉藤崇史
202006050811,2,7,14,サラキア,牝5,55.0,松山弘平,2:35.0,クビ,74.9,11.0,448(-2),[西] 池添学
202006050811,3,7,13,フィエールマン,牡5,57.0,ルメール,2:35.1,クビ,3.5,2.0,478(0),[東] 手塚貴久
202006050811,4,4,7,ラッキーライラック,牝5,55.0,福永祐一,2:35.5,2.1/2,8.1,4.0,522(0),[西] 松永幹夫
202006050811,5,3,5,ワールドプレミア,牡4,57.0,武豊,2:35.6,1/2,13.5,5.0,484(+2),[西] 友道康夫
202006050811,5,5,10,カレンブーケドール,牝4,55.0,池添謙一,2:35.6,同着,7.9,3.0,478(+4),[東] 国枝栄
202006050811,7,4,8,ペルシアンナイト,牡6,57.0,大野拓弥,2:35.6,アタマ,148.1,12.0,500(0),[西] 池江泰寿
202006050811,8,2,3,クレッシェンドラヴ,牡6,57.0,坂井瑠星,2:35.8,1.1/2,188.1,16.0,500(0),[東] 林徹
202006050811,9,8,15,オセアグレイト,牡4,57.0,横山典弘,2:35.9,クビ,183.4,15.0,494(-2),[東] 菊川正達
202006050811,10,2,4,ラヴズオンリーユー,牝4,55.0,Ｍ．デム,2:35.9,クビ,15.5,6.0,480(-6),[西] 矢作芳人
